In [1]:
import copy
import random
import torch
from torch.autograd import Variable
import matplotlib.pyplot as plt
from PIL import Image
from IPython import display
import math
import torchvision.transforms as T
import numpy as np
import time
import socket

In [2]:
class DQN():
    ''' Deep Q Neural Network class. '''
    def __init__(self, state_dim, action_dim, hidden_dim=64, lr=0.05):
        self.criterion = torch.nn.MSELoss()
        self.model = torch.nn.Sequential(
                        torch.nn.Linear(state_dim, hidden_dim),
                        torch.nn.LeakyReLU(),
                        torch.nn.Linear(hidden_dim, hidden_dim*2),
                        torch.nn.LeakyReLU(),
                        torch.nn.Linear(hidden_dim*2, action_dim))
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr) #cambiar
        
    def update(self, state, y):
        """Update the weights of the network given a training sample. """
        tensor = torch.Tensor(state)
        y_pred = self.model(tensor)
        loss = self.criterion(y_pred, Variable(torch.Tensor(y)))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
    def predict(self, state):
        """ Compute Q values for all actions using the DQL. """
        with torch.no_grad():
            return self.model(torch.Tensor(state))

state_dim -> input red nuronal, depende del estado

action_dim -> 4 acciones: up, down, left y right

hidden_dim -> ajustar hiperparámetro

lr -> ajustar hiperparámetro

In [3]:
action_dim = 4
hidden_dim = 100
lr = 0.05

### Prueba preliminar
Distancia a la pill más cercana en cada una de las direcciones.

In [4]:
state_dim = 4

In [ ]:
episodes = 100
gamma = 0.7

In [5]:
model = DQN(state_dim, action_dim, hidden_dim, lr)

In [ ]:
class Game():
    def __init__(self, host="localhost", port=38514):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        try:
            self.sock.bind((host, port))
        except socket.error as err:
            print('Bind failed. Error Code : ' .format(err))
        
    def connect():
        self.sock.listen(1)
        self.conn, _ = self.sock.accept()
        
    def get_state():
        return self.conn.recv(512)
        
    def step(action):
        self.conn.send(bytes(str(action) + "\n",'UTF-8'))
        data = self.conn.recv(512)
        data = data.decode(encoding='UTF-8')
        return data.split(";")

In [ ]:
game = Game()

In [ ]:
def q_learning(game, model, episodes, gamma=0.7, title = 'DQN'):
    """Deep Q Learning algorithm using the DQN. """
    final = []
    episode_i=0
    for episode in range(episodes):
        episode_i+=1
                
        # Reset state
        state = game.get_state()
        done = False
        total = 0
        
        while not done:
            # Implement greedy search policy to explore the state space
            q_values = model.predict(state) #Devuelve array con probabilidades de accion
            action = torch.argmax(q_values).item() #Accion a realizar
            
            # Take action and add reward to total
            next_state, reward, done = game.step(action) #Quizas reward requiera tratamiento, 
            
            # Update total and memory
            total += reward
            q_values = q_values.tolist()
             
            if done:
                q_values[action] = reward
                # Update network weights
                model.update(state, q_values)
                break
                
            # Update network weights using the last step only
            q_values_next = model.predict(next_state)
            q_values[action] = reward + gamma * torch.max(q_values_next).item()
            model.update(state, q_values)

            state = next_state
        
        final.append(total)
        plot_res(final, title)
        
        print("episode: {}, total reward: {}".format(episode_i, total))
            
        #if total == 500:
            #torch.save(model, "model.mdl")
            #return final
    return final

In [ ]:
def plot_res(values, title=''):   
    ''' Plot the reward curve and histogram of results over time.'''
    # Update the window after each episode
    clear_output(wait=True)
    
    # Define the figure
    f, ax = plt.subplots(nrows=1, ncols=2, figsize=(12,5))
    f.suptitle(title)
    ax[0].plot(values, label='score per run')
    ax[0].axhline(195, c='red',ls='--', label='goal')
    ax[0].set_xlabel('Episodes')
    ax[0].set_ylabel('Reward')
    x = range(len(values))
    ax[0].legend()
    # Calculate the trend
    try:
        z = np.polyfit(x, values, 1)
        p = np.poly1d(z)
        ax[0].plot(x,p(x),"--", label='trend')
    except:
        print('')
    
    # Plot the histogram of results
    ax[1].hist(values[-50:])
    ax[1].axvline(195, c='red', label='goal')
    ax[1].set_xlabel('Scores per Last 50 Episodes')
    ax[1].set_ylabel('Frequency')
    ax[1].legend()
    plt.show()